In [17]:
import numpy as np
import sympy as sp
from sympy import *


Parametros da matriz geradora.


In [18]:
Mod = 11 
def inverse_mod(number, mod = Mod):
    for value in range(1, mod):
        if (   (  ( number % mod   )  * ( value % mod) ) % mod == 1   ):
            return value
def zeros_matrix(rows, cols):
   
    M = []
    while len(M) < rows:
        M.append([])
        while len(M[-1]) < cols:
            M[-1].append(0.0)

    return M
def matrix_multiply(matrix_A,matrix_B,mod= Mod):
   
    rowsA = len(matrix_A)
    colsA = len(matrix_A[0])

    rowsB = len(matrix_B)
    colsB = len(matrix_B[0])

    if colsA != rowsB:
        raise ArithmeticError('Number of A columns must equal number of B rows.')

    C = zeros_matrix(rowsA, colsB)

    for i in range(rowsA):
        for j in range(colsB):
            total = 0
            for ii in range(colsA):
                total += matrix_A[i][ii] * matrix_B[ii][j] 
            C[i][j] = total % mod

    return C
def delete_row_column(matrix, i, j):
    rows = len(matrix)
    columns = len(matrix[0])
    return [[matrix[row][col] for col in range(columns) if col != j] for row in range(rows) if row != i]
def minor(matrix, i, j):
    matrix = delete_row_column(matrix, i, j)
    return determinant(matrix)
def determinant(matrix, mod= Mod):
    assert len(matrix) == len(matrix[0]), "matrix should be square"
    order = len(matrix)
    if order == 1:
        return matrix[0][0] % mod
    elif order == 2:
        det = (matrix[0][0] * matrix[1][1]) - (matrix[0][1] * matrix[1][0])  # ad - bc
        return det % mod
    else:
        det = 0
        for i in range(order):
            coefficient = matrix[0][i]
            if i % 2 == 1:
                coefficient *= -1
            det += coefficient * minor(matrix, 0, i)
        return det%Mod
def transpose(matrix, mod = Mod):
    rows = len(matrix)
    columns = len(matrix[0])
    return [[matrix[row][col] % mod for row in range(rows)] for col in range(columns)]
def inverse_matrix(matrix, mod = 11):
    assert len(matrix) == len(matrix[0]), "matrix should be square"
    order = len(matrix)

    det = determinant(matrix)
    cofactors = []
    for i in range(order):
        cofactors_row = []
        for j in range(order):
            coefficient = -1 if (i % 2) ^ (j % 2) else 1  # For the chequered pattern of + and -
            cofactors_row.append((coefficient % mod) * minor(matrix, i, j))
        cofactors.append(cofactors_row)

    adjugate = transpose(cofactors)
    inv_mod = inverse_mod (det) 
    inverse = [[(element * inv_mod ) % mod for element in row] for row in adjugate]  # Divide by determinant(Multiplicar pela inversa do determinate)
    return inverse
def is_echelon_form(matrix_rref):
    diagonal = list()
    for i in range(k):
        # verificar as diagonais
        diagonal.append( matrix_rref[i,i] )
    if sum(diagonal) == k:
        return True
    else:
        return False
def Generate_S_invertible(G):
    np.random.seed(109557)
    condition = True
    dim = G.shape[0]
    while condition:
        S = np.mod(np.random.permutation( dim * dim   ).reshape(dim  , dim ),Mod)
        if np.linalg.det(S)  !=0:
            condition = False
    return  S
def generate_permutation_matrix(G):
    # Generate a random permutation of the indices
    permutation = np.random.permutation(G.shape[1])
    # Create a permutation matrix using the permutation of indices
    permutation_matrix = np.eye(G.shape[1], dtype=int)[permutation]
    return permutation_matrix.astype(int)
def Transform_echelon_Matrix_mod(x, Mod):
    numer, denom = x.as_numer_denom()
    return numer * sp.mod_inverse(denom, Mod) % Mod
def echelon_form(matrix):
    matrix = sp.Matrix(matrix)
    #Find row-reduced echolon form of matrix modulo 13:
    matrix_rref = matrix.rref( iszerofunc = lambda x: x % Mod ==0)[0].applyfunc(lambda x:  Transform_echelon_Matrix_mod(x,Mod)   )
    row, cols =  matrix_rref.shape  
    if is_echelon_form(matrix_rref):
        return np.array( matrix_rref ).astype(int), True
    else:
        return np.array( matrix_rref ).astype(int),False

In [19]:
alpha = np.array([8, 2 , 1,  7, 9, 3, 6, 4, 5])
beta  = np.array([9, 6,  8 , 5, 2, 10, 7, 2, 7])
message = [5, 4, 1, 3]
t = 3



error = np.zeros(len(beta ), dtype = int)
for i in range(len(beta)):
    if i < t:
        aux = np.random.randint(0, Mod)
        if error[i] == 0:
            error[i] = aux
        else:
            error[i] = 0
    pass
n = len(alpha)
k = len(message)
np.random.shuffle(error)

 

In [20]:


Generator_Matrix = np.zeros((k,n), dtype = int)
message = np.array([ 4,5,2,8] ) 
for i in range(k):
    for j in range(n):
        Generator_Matrix[i, j] = np.mod((int((beta[j]*(alpha[j])**i))), Mod) 



In [21]:
Public_key  = np.mod( np.dot(np.mod(np.dot(Generate_S_invertible(Generator_Matrix), Generator_Matrix), Mod) , generate_permutation_matrix(Generator_Matrix)), Mod)
c =   np.mod(np.dot(message ,Public_key ), Mod) 
y = c + error
y
 


array([11,  0,  6,  8,  4,  7,  5,  7,  6])

In [22]:
Echelon_Matrix,_ = echelon_form(Public_key)
Echelon_Matrix
 

array([[ 1,  0,  0,  0,  2, 10,  8,  2,  5],
       [ 0,  1,  0,  0,  4,  3,  1,  9,  4],
       [ 0,  0,  1,  0,  6,  1,  6,  4,  9],
       [ 0,  0,  0,  1,  6,  6, 10,  1,  1]])

Encontrar os valores de alfas.
| $d$ | $\bar{\alpha}_1$ | $\bar{\alpha}_2$ | $\bar{\alpha}_3$ | $\bar{\alpha}_4$ | $\bar{\alpha}_5$ | $\bar{\alpha}_6$ | $\bar{\alpha}_7$ | $\bar{\alpha}_8$ | $\bar{\alpha}_9$ |
|-----|------------------|------------------|------------------|------------------|------------------|------------------|------------------|------------------|------------------|
| 1   | 0                | 1                | 10               | 5                | 2                | 9                | 3                | 6                | 7                |
| 2   | 0                | 1                | 10               | 5                | 5                | 4                | 7                | 10               | 10               |


In [23]:
alphas_list = {"d1": [0,1,10,5,2,9,3,6,7], "d2": [0,1,10,5,5,4,7,10,10]}
y = np.mod( np.dot(message,Public_key ), Mod)
c, alpha = [ y[i] for i in range(k + 1) ], list()
for i in range( len(alphas_list["d1"])):
    if i < k + 1:
       alpha.append(  (alphas_list["d1"])[i] )
alpha, y
         
    

([0, 1, 10, 5, 2], array([4, 0, 6, 8, 4, 6, 5, 7, 5]))

In [24]:
"""  k + 1 colunas da chave publica"""
M_linha_1  = np.zeros( (k, k + 1) ).astype(int) 
for rows in range(k):
    for cols in range(k + 1):
        M_linha_1[rows, cols] = Public_key[rows, cols]

M_linha_1

array([[ 9,  8,  0,  2,  7],
       [ 1,  8,  3,  7,  6],
       [ 1,  2,  5,  1,  2],
       [ 2,  7, 10,  5,  1]])

9c1 + 8c2+  0c3 +  2c4 +  7c5 = 0,
1c1 + 8c2+  3c3 +  7c4 +  6c5 = 0,
1c1 + 2c2+  5c3 +  1c4 +  2c5 = 0,
2c1 + 7c2+ 10c3 +  5c4 +  1c5 = 0


Encontrar a solução trivial de M'c = 0


Supondo que c1 = 1

In [25]:
M = np.array([[8,  0,  2, 7],
             [8,  3,  7, 6],
             [2,  5,  1, 2], 
             [7,  10, 5, 1]])

c = np.array([ [-9%11],[-1%11],[-1%11],[-2%11]

])
sol = np.mod(np.dot(np.mod(inverse_matrix(M), Mod), c), Mod)

solution_c = np.zeros(k + 1).astype(int)
solution_c[0] = 1
for index, i in enumerate(sol):
    solution_c[index + 1] = i[0]  
print(solution_c)

[1 2 3 3 5]


In [26]:
alpha
G_linha1 = np.zeros((k,k+1)).astype(int) 
for i in range(k):
    for j in range(k+1):
        G_linha1[i, j] = np.mod((int((solution_c[j]*(alpha[j])**i))), Mod) 
G_linha1

array([[ 1,  2,  3,  3,  5],
       [ 0,  2,  8,  4, 10],
       [ 0,  2,  3,  9,  9],
       [ 0,  2,  8,  1,  7]])

In [27]:
A = np.array([[ 2,  3,  3,  5],
             [ 2,  8,  4, 10],
             [ 2,  3,  9,  9],
             [ 2,  8,  1,  7]])
b = np.array([[10],[0],[0],[0]])


sol = np.mod(np.dot(np.mod(inverse_matrix(A), Mod), b), Mod)



betas_novo = np.zeros(k + 1).astype(int)
betas_novo[0] = 1
for index, i in enumerate(sol):
    betas_novo[index + 1] = i[0]  
print(betas_novo)





[1 9 5 6 5]


In [28]:
G_tilda1 = np.zeros((k,k +1)).astype(int) 
for i in range(k):
    for j in range(k + 1):
        G_tilda1[i, j] = np.mod((int((betas_novo[j]*(alpha[j])**i))), Mod) 

G_tilda1

array([[ 1,  9,  5,  6,  5],
       [ 0,  9,  6,  8, 10],
       [ 0,  9,  5,  7,  9],
       [ 0,  9,  6,  2,  7]])

In [29]:
G_tilda2 = G_tilda1[:,:k]
M_linha2 = Public_key[:,:k]
G_tilda2 = G_tilda1[:,:k]
Matrix_S = np.mod(  np.dot(M_linha2, inverse_matrix(G_tilda2, Mod))   ,Mod) 

Matrix_S
 

array([[ 9,  3,  0,  6],
       [ 1,  5,  5,  7],
       [ 1,  8, 10,  2],
       [ 2,  8,  0,  3]])

In [30]:
G_tilda= np.mod(np.dot(inverse_matrix(Matrix_S, Mod),Public_key), Mod)
G_tilda

np.mod(    transpose(G_tilda), Mod)

array([[ 1,  0,  0,  0],
       [ 9,  9,  9,  9],
       [ 5,  6,  5,  6],
       [ 6,  8,  7,  2],
       [ 5, 10,  9,  7],
       [ 1,  4,  8,  1],
       [ 8,  4, 10, 10],
       [10,  3,  9,  8],
       [ 4, 10,  0,  4]])

In [31]:
Public_key_recovery = np.mod(np.dot(Matrix_S, G_tilda ),Mod)
Public_key_recovery

array([[ 9,  8,  0,  2,  7,  5,  1,  4,  2],
       [ 1,  8,  3,  7,  6,  2,  5,  5,  5],
       [ 1,  2,  5,  1,  2,  5,  6,  8,  4],
       [ 2,  7, 10,  5,  1,  4,  1,  2,  1]])

In [32]:
for i, j in zip(Public_key_recovery, Public_key):
    print(i , "\t\t", j)

[9 8 0 2 7 5 1 4 2] 		 [9 8 0 2 7 5 1 4 2]
[1 8 3 7 6 2 5 5 5] 		 [1 8 3 7 6 2 5 5 5]
[1 2 5 1 2 5 6 8 4] 		 [1 2 5 1 2 5 6 8 4]
[ 2  7 10  5  1  4  1  2  1] 		 [ 2  7 10  5  1  4  1  2  1]
